In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt

In [4]:
loan_train=pd.read_csv('train_u6lujuX_CVtuZ9i.csv')
print(loan_train.shape)
loan_train.head()

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
total_null=loan_train.isnull().sum().sort_values(ascending=False)
total_null.head(10)

Credit_History      50
Self_Employed       32
LoanAmount          22
Dependents          15
Loan_Amount_Term    14
Gender              13
Married              3
Loan_ID              0
Education            0
ApplicantIncome      0
dtype: int64

In [7]:
loan_train['Gender']=loan_train['Gender'].fillna(loan_train['Gender'].dropna().mode().values[0])
loan_train['Married']=loan_train['Married'].fillna(loan_train['Married'].dropna().mode().values[0])
loan_train['Dependents']=loan_train['Dependents'].fillna(loan_train['Dependents'].dropna().mode().values[0])
loan_train['Self_Employed']=loan_train['Self_Employed'].fillna(loan_train['Self_Employed'].dropna().mode().values[0])
loan_train['LoanAmount']=loan_train['LoanAmount'].fillna(loan_train['LoanAmount'].dropna().mean())
loan_train['Loan_Amount_Term']=loan_train['Loan_Amount_Term'].fillna(loan_train['Loan_Amount_Term'].dropna().mode().values[0])
loan_train['Credit_History']=loan_train['Credit_History'].fillna(loan_train['Credit_History'].dropna().mode().values[0])

In [8]:
loan_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         614 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [9]:
print(set(loan_train['Gender'].values.tolist()))
print(set(loan_train['Dependents'].values.tolist()))
print(set(loan_train['Married'].values.tolist()))
print(set(loan_train['Education'].values.tolist()))
print(set(loan_train['Self_Employed'].values.tolist()))
print(set(loan_train['Loan_Status'].values.tolist()))
print(set(loan_train['Property_Area'].values.tolist()))

{'Male', 'Female'}
{'0', '2', '3+', '1'}
{'No', 'Yes'}
{'Graduate', 'Not Graduate'}
{'No', 'Yes'}
{'Y', 'N'}
{'Rural', 'Urban', 'Semiurban'}


In [13]:
loan_train['Loan_Status']=loan_train['Loan_Status'].map({'N':0, 'Y':1}).astype(int)
loan_train=pd.get_dummies(loan_train, columns=['Gender','Dependents','Married','Education','Self_Employed','Property_Area'])



In [14]:
standardScaler=StandardScaler()
columns_to_scale=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']
loan_train[columns_to_scale]=standardScaler.fit_transform(loan_train[columns_to_scale])

In [16]:
y=loan_train['Loan_Status']
x=loan_train.drop(['Loan_Status', 'Loan_ID'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2, random_state= 42)

In [19]:
from sklearn.model_selection import RandomizedSearchCV

In [20]:
rf_param_grid={'n_estimators':range(1,1000,100)}
rf=RandomForestClassifier()
rf_random=RandomizedSearchCV(param_distributions=rf_param_grid, estimator=rf,scoring='accuracy', verbose=0, n_iter=100, cv=4)
rf_random.fit(x_train,y_train)
best_params=rf_random.best_params_
print(f'Best parameters:{best_params}')
y_pred1=rf_random.predict(x_test)
print(f'Accuracy:{np.sum(y_pred1==y_test)/len(y_test)}')


C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters:{'n_estimators': 601}
Accuracy:0.7723577235772358
